# 🚀 Backend ChatGPT - Jupyter Notebook
Backend completo para conectar con la API de ChatGPT

## 1. Instalación de dependencias

In [2]:
# !pip install fastapi uvicorn openai python-dotenv requests

## 2. Configuración y variables

In [3]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv

# Cargar variables de entorno
load_dotenv()

# Configuración
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") or input("Ingresa tu API Key de OpenAI: ")
PORT = 8000

# Inicializar cliente OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

print("✅ Configuración cargada correctamente")
print(f"🔑 API Key: {'✅ Configurada' if OPENAI_API_KEY else '❌ No configurada'}")
print(f"🌐 Puerto: {PORT}")

✅ Configuración cargada correctamente
🔑 API Key: ✅ Configurada
🌐 Puerto: 8000


## 3. Clases y modelos de datos

In [4]:
from pydantic import BaseModel

class QuestionRequest(BaseModel):
    question: str
    model: str = "gpt-3.5-turbo"
    max_tokens: int = 500
    temperature: float = 0.7

class QuestionResponse(BaseModel):
    answer: str
    model: str
    tokens_used: int

print("✅ Modelos de datos definidos")

✅ Modelos de datos definidos


## 4. Función principal para ChatGPT

In [5]:
def ask_chatgpt(question: str, model: str = "gpt-3.5-turbo", max_tokens: int = 500, temperature: float = 0.7):
    """
    Función para hacer preguntas a la API de ChatGPT
    """
    try:
        print(f"🤖 Enviando pregunta a {model}...")
        
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "Eres un asistente útil y amigable."},
                {"role": "user", "content": question}
            ],
            max_tokens=max_tokens,
            temperature=temperature
        )
        
        answer = response.choices[0].message.content
        tokens_used = response.usage.total_tokens
        
        print(f"✅ Respuesta recibida ({tokens_used} tokens)")
        
        return QuestionResponse(
            answer=answer,
            model=model,
            tokens_used=tokens_used
        )
        
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return None

## 5. Prueba directa de la función

In [6]:
# Prueba directa sin servidor
test_question = "Explique qué es la inteligencia artificial en 2 líneas"

result = ask_chatgpt(test_question)

if result:
    print("\n" + "="*50)
    print("📝 PREGUNTA:")
    print(test_question)
    print("\n💬 RESPUESTA:")
    print(result.answer)
    print("\n📊 METADATA:")
    print(f"Modelo: {result.model}")
    print(f"Tokens usados: {result.tokens_used}")
    print("="*50)

🤖 Enviando pregunta a gpt-3.5-turbo...
✅ Respuesta recibida (89 tokens)

📝 PREGUNTA:
Explique qué es la inteligencia artificial en 2 líneas

💬 RESPUESTA:
La inteligencia artificial es la capacidad de las máquinas para realizar tareas que normalmente requieren inteligencia humana, como el aprendizaje, la resolución de problemas y la toma de decisiones, mediante algoritmos y datos.

📊 METADATA:
Modelo: gpt-3.5-turbo
Tokens usados: 89


## 6. Backend FastAPI (ejecutable desde notebook)

In [7]:
from fastapi import FastAPI
import uvicorn
import threading

# Crear aplicación FastAPI
app = FastAPI(
    title="ChatGPT API Backend",
    description="Backend para conectar con ChatGPT",
    version="1.0.0"
)

@app.get("/")
async def root():
    return {"message": "ChatGPT API Backend funcionando desde Jupyter"}

@app.get("/health")
async def health_check():
    return {"status": "healthy", "service": "chatgpt-backend"}

@app.post("/ask")
async def ask_question(request: QuestionRequest):
    result = ask_chatgpt(
        question=request.question,
        model=request.model,
        max_tokens=request.max_tokens,
        temperature=request.temperature
    )
    
    if result:
        return result
    else:
        return {"error": "No se pudo obtener respuesta de ChatGPT"}

def run_server():
    uvicorn.run(app, host="0.0.0.0", port=PORT)

print("✅ Aplicación FastAPI creada")

✅ Aplicación FastAPI creada


## 7. Ejecutar servidor en segundo plano

In [ ]:
# Ejecutar el servidor en un hilo separado
server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

print(f"🚀 Servidor iniciado en http://localhost:{PORT}")
print("📋 Endpoints disponibles:")
print(f"   GET  http://localhost:{PORT}/")
print(f"   GET  http://localhost:{PORT}/health")
print(f"   POST http://localhost:{PORT}/ask")
print("\n⚠️  El servidor se detendrá cuando cierres el notebook")

🚀 Servidor iniciado en http://localhost:8000
📋 Endpoints disponibles:
   GET  http://localhost:8000/
   GET  http://localhost:8000/health
   POST http://localhost:8000/ask

⚠️  El servidor se detendrá cuando cierres el notebook


INFO:     Started server process [22584]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 8000): solo se permite un uso de cada dirección de socket (protocolo/dirección de red/puerto)
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


## 8. Prueba del servidor desde el notebook

In [9]:
import requests
import time

# Esperar a que el servidor esté listo
time.sleep(2)

def test_server():
    base_url = f"http://localhost:{PORT}"
    
    print("🧪 Probando servidor...\n")
    
    # Probar health check
    try:
        health_response = requests.get(f"{base_url}/health")
        print(f"✅ Health check: {health_response.json()}")
    except:
        print("❌ Health check falló")
        return
    
    # Probar endpoint de pregunta
    test_data = {
        "question": "¿Cuál es la capital de España?",
        "model": "gpt-3.5-turbo",
        "max_tokens": 100,
        "temperature": 0.7
    }
    
    try:
        response = requests.post(f"{base_url}/ask", json=test_data)
        
        if response.status_code == 200:
            result = response.json()
            print(f"✅ Pregunta respondida correctamente")
            print(f"📝 Respuesta: {result['answer']}")
            print(f"🔧 Modelo: {result['model']}")
            print(f"🔢 Tokens: {result['tokens_used']}")
        else:
            print(f"❌ Error en la petición: {response.status_code}")
            print(response.text)
            
    except Exception as e:
        print(f"❌ Error al conectar con el servidor: {e}")

# Ejecutar prueba
test_server()

🧪 Probando servidor...

✅ Health check: {'status': 'healthy', 'service': 'chatgpt-backend'}
✅ Pregunta respondida correctamente
📝 Respuesta: La capital de España es Madrid. Es una ciudad muy importante y con una gran riqueza cultural, histórica y artística. ¿Hay algo más en lo que pueda ayudarte?
🔧 Modelo: gpt-3.5-turbo
🔢 Tokens: 71


## 9. Interfaz interactiva para hacer preguntas

In [ ]:
from IPython.display import display
import ipywidgets as widgets

def create_chat_interface():
    """Interfaz gráfica para hacer preguntas"""
    
    # Widgets
    question_input = widgets.Textarea(
        value='',
        placeholder='Escribe tu pregunta aquí...',
        description='Pregunta:',
        layout=widgets.Layout(width='80%', height='100px')
    )
    
    model_dropdown = widgets.Dropdown(
        options=['gpt-3.5-turbo', 'gpt-4', 'gpt-4-turbo-preview'],
        value='gpt-3.5-turbo',
        description='Modelo:'
    )
    
    ask_button = widgets.Button(
        description='Preguntar a ChatGPT',
        button_style='success',
        tooltip='Hacer pregunta'
    )
    
    output = widgets.Output()
    
    def on_ask_button_clicked(b):
        with output:
            output.clear_output()
            
            if not question_input.value.strip():
                print("❌ Por favor escribe una pregunta")
                return
            
            print("⏳ Procesando pregunta...")
            
            result = ask_chatgpt(
                question=question_input.value,
                model=model_dropdown.value
            )
            
            if result:
                print("\n" + "="*60)
                print("💬 RESPUESTA:")
                print(result.answer)
                print("\n📊 Información:")
                print(f"   Modelo: {result.model}")
                print(f"   Tokens usados: {result.tokens_used}")
                print("="*60)
    
    ask_button.on_click(on_ask_button_clicked)
    
    # Display interface
    display(widgets.VBox([
        widgets.HTML("<h3>🤖 Chat con ChatGPT</h3>"),
        question_input,
        model_dropdown,
        ask_button,
        output
    ]))

# Mostrar interfaz interactiva
create_chat_interface()

## 10. Guardar configuración

In [ ]:

print("✅ Archivo .env creado/actualizado")
print("📁 El notebook está listo para usar")

✅ Archivo .env creado/actualizado
📁 El notebook está listo para usar
